In [6]:
"""
Train on images split into directories. This assumes we've split
our videos into frames and moved them to their respective folders.
Use keras 2+ and tensorflow 1+
Based on:
https://keras.io/preprocessing/image/
and
https://keras.io/applications/
"""
import os
from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet50 import ResNet50
from keras.optimizers import SGD, Adagrad, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Reshape, LSTM, Dropout, Average
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
import numpy as np
#from UCFdata import DataSet




In [7]:
# def get_generators():
#     train_generator = generator_shuffle_by_class(
#           'object_data/train',
#           batch_size=batch_size,target_size = (224,224))
    
    
#     validation_generator = generator_shuffle_by_class(
#           'object_data/val',
#           batch_size=batch_size,target_size = (224,224))
#     return train_generator, validation_generator

In [8]:
def get_model(weights='imagenet'):
    # create the base pre-trained model

                    
    base_model = InceptionV3(weights=weights, include_top=False)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024,activation = 'relu')(x)
    x = Dropout(0.9)(x)
    predictions = Dense(4, activation='softmax')(x)

    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)

    for layer in base_model.layers:
        layer.trainable = False

    # compile the model (should be done *after* setting layers to non-trainable)
    model.compile(optimizer=Adagrad(lr=0.00005, decay = 1e-6), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [9]:
def get_generators():
    datagen = ImageDataGenerator(rescale=1. / 255)
    train_generator = datagen.flow_from_directory(
          'data/train',
          target_size=(224, 224),
          batch_size=batch_size,
          class_mode='categorical',  # this means our generator will only yield batches of data, no labels
          shuffle=True,
          classes=['angry','happy', 'sad', 'submissive'])

    validation_generator = datagen.flow_from_directory(
          'data/val',
          target_size=(224, 224),
          batch_size=batch_size,
          class_mode='categorical',  # this means our generator will only yield batches of data, no labels
          shuffle=True,
          classes=['angry','happy','sad', 'submissive'])
    return train_generator, validation_generator


In [10]:
def train_model(model, nb_epoch, generators, callbacks=[]):
    train_generator, validation_generator = generators
    traindata = 23750 
    valdata = 7100 
    model.fit_generator(
        train_generator,
        steps_per_epoch=traindata/batch_size,
        validation_data=validation_generator,
        validation_steps=valdata/batch_size,
        epochs=nb_epoch,
        callbacks=callbacks)
    return model


In [11]:
def main(weights_file):
    model = get_model()
    generators = get_generators()    
    print("Training Mid layers")
    model.summary()
    model = train_model(model, 10, generators,callbacks_list)

In [12]:

batch_size = 25
# Helper: Save the min val_loss model in each epoch.

checkpointer = ModelCheckpoint(
    filepath='incept(global)_latest.{epoch:03d}-{val_loss:.2f}-{val_acc:.2f}.hdf5', 
    #save format is epoch-loss-acc-time-date(mmddyyyy)
    verbose=1,
    monitor='val_acc',
    save_best_only=True)

# Helper: Stop when we stop learning.
# patience: number of epochs with no improvement after which training will be stopped.
early_stopper = EarlyStopping(patience=25)

# Helper: TensorBoard
#tensorboard = TensorBoard(log_dir='utils/logs/')
callbacks_list = [checkpointer]
if __name__ == '__main__':
    weights_file = None
    main(weights_file)

Found 23750 images belonging to 4 classes.
Found 7100 images belonging to 4 classes.
Training Mid layers
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
____

Epoch 1/10
950/950 [==============================] - 285s 300ms/step - loss: 1.6213 - acc: 0.3313 - val_loss: 1.4563 - val_acc: 0.3148

Epoch 00001: val_acc improved from -inf to 0.31479, saving model to incept(global)_latest.001-1.46-0.31.hdf5
Epoch 2/10
950/950 [==============================] - 250s 263ms/step - loss: 1.4135 - acc: 0.3928 - val_loss: 1.4121 - val_acc: 0.3232

Epoch 00002: val_acc improved from 0.31479 to 0.32324, saving model to incept(global)_latest.002-1.41-0.32.hdf5
Epoch 3/10
950/950 [==============================] - 266s 280ms/step - loss: 1.3056 - acc: 0.4300 - val_loss: 1.3907 - val_acc: 0.3273

Epoch 00003: val_acc improved from 0.32324 to 0.32732, saving model to incept(global)_latest.003-1.39-0.33.hdf5
Epoch 4/10
129/950 [===>..........................] - ETA: 2:49 - loss: 1.2877 - acc: 0.4338

KeyboardInterrupt: 

In [13]:
from keras.models import load_model
model = load_model('incept(global)_latest.002-1.41-0.32.hdf5')

In [14]:
"""
Classify test images set through our CNN.
Use keras 2+ and tensorflow 1+
It takes a long time for hours.
"""
import numpy as np
import operator
import random
import glob
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator


# CNN model evaluate

test_data_gen = ImageDataGenerator(rescale=1. / 255)
batch_size = 25
test_path = 'object_data/test'  #path to your validation / test videos
test_generator = test_data_gen.flow_from_directory(test_path, target_size=(224, 224),
                                                   batch_size=batch_size, classes=['angry','happy','sad', 'submissive'],
                                                   class_mode='categorical',shuffle = False)

Found 14950 images belonging to 4 classes.


In [15]:
test_data_num = 14950         
test_generator.reset() #to avoid having bugs in generator.
# if you dont invoke .reset(), it will starts to mix the order of the array from predicted generator

predicted_array = model.predict_generator(test_generator, steps = test_data_num/batch_size)
len(predicted_array) #check if generated data is the same as test_data_num

14950

In [16]:
frame = [] #check all frame names for each validation / test videos
filenames = [] #check all filenames for each validation / test videos
gt = [] #ground truth storage for each validation / test videos
length = [] #how many frames belongs to which test / validation videos
final_predicted_array = [] #final predicted results for all videos

catfiles = os.listdir(test_path)
for i in range(len(catfiles)):
    subvideos = os.listdir(os.path.join(test_path,catfiles[i]))
    filenames.extend(subvideos)
    for j in range(len(subvideos)):
        contentvideos = os.listdir(os.path.join(test_path,catfiles[i],subvideos[j]))
        frame.append(contentvideos)
        length.append(len(contentvideos))
        gt.append(i)
cum_length = np.cumsum(length)
test2 = predicted_array[:cum_length[0]]
final_predicted_array.append(test2)
for i in range(len(cum_length)-1):
    test2 = predicted_array[cum_length[i]:cum_length[i+1]]
    final_predicted_array.append(test2)
votepred = []
meanpred = []

for i in range(len(final_predicted_array)):
    votectg = np.bincount(np.argmax(final_predicted_array[i], axis=1))
    votepred.append(np.argmax(votectg))
    meanctg = np.argmax(np.mean(final_predicted_array[i], axis=0))
    meanpred.append(meanctg)

vote = 0
mean = 0
for i in range(len(gt)):
    if(gt[i] == votepred[i]):
        vote += 1
    if(gt[i] == meanpred[i]):
        mean += 1
        
vote_acc = vote/len(gt)
mean_acc = mean/len(gt)

print("voting accuracy : ", vote_acc)
print("mean accuracy : ", mean_acc)

voting accuracy :  0.2809364548494983
mean accuracy :  0.2842809364548495
